In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import Xception, MobileNetV3Large
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Ruta y carga de datos
ruta_base = '/content/drive/MyDrive/SCUBI DU'
def is_valid_image(file_path):
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    return any(file_path.lower().endswith(ext) for ext in valid_extensions)

clases_unicas = sorted([d for d in os.listdir(ruta_base) if os.path.isdir(os.path.join(ruta_base, d))])
class_to_label = {name: idx for idx, name in enumerate(clases_unicas)}

imagenes_cargadas, etiquetas_cargadas = [], []
for class_name in clases_unicas:
    class_folder = os.path.join(ruta_base, class_name)
    valid_images = [os.path.join(class_folder, f) for f in os.listdir(class_folder) if is_valid_image(f)]
    for image_path in valid_images:
        img = load_img(image_path, target_size=(224, 224))
        imagenes_cargadas.append(np.array(img))
        etiquetas_cargadas.append(class_to_label[class_name])

X = np.array(imagenes_cargadas) / 255.0
y_raw = np.array(etiquetas_cargadas)
y = to_categorical(y_raw)

# Parámetros óptimos fijos
parametros_fijos = {
    'batch_size': 6,
    'activation': 'sigmoid',
    'patience': 3,
    'epochs': 10,
    'learning_rate': 0.0001
}

# Configuraciones de capas
layer_unit_configs = {
    1: [[64], [128], [256]],
    2: [[256, 128], [128, 64], [64, 32]],
    3: [[256, 128, 64], [128, 64, 32], [64, 32, 16]]
}

# Valores de tolerancia
modelos = {
    'Xception': Xception,
    'MobileNetV3Large': MobileNetV3Large
}
tolerancias = [0.005, 0.02]

# Función para construir el modelo
def build_model(base_model_fn, units_per_layer_list, activation='relu', learning_rate=0.001):
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    for units in units_per_layer_list:
        model.add(layers.Dense(units, activation=activation))
        model.add(layers.Dropout(0.5))
    model.add(layers.Dense(y.shape[1], activation='softmax'))
    opt = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Validación cruzada 5-fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
resultados = []

for nombre_modelo, modelo_base in modelos.items():
    for tolerancia in tolerancias:
        for num_layers, configs in layer_unit_configs.items():
            for units_list in configs:
                f1_macro_list, f1_weighted_list, precision_list, recall_list = [], [], [], []

                for train_index, test_index in skf.split(X, y_raw):
                    X_train, X_test = X[train_index], X[test_index]
                    y_train, y_test = y[train_index], y[test_index]

                    model = build_model(
                        base_model_fn=modelo_base,
                        units_per_layer_list=units_list,
                        activation=parametros_fijos['activation'],
                        learning_rate=parametros_fijos['learning_rate']
                    )
                    es = callbacks.EarlyStopping(
                        monitor='val_loss',
                        patience=parametros_fijos['patience'],
                        min_delta=tolerancia,
                        restore_best_weights=True
                    )
                    model.fit(
                        X_train, y_train,
                        epochs=parametros_fijos['epochs'],
                        batch_size=parametros_fijos['batch_size'],
                        validation_split=0.2,
                        callbacks=[es],
                        verbose=0
                    )

                    y_pred = model.predict(X_test)
                    y_true = np.argmax(y_test, axis=1)
                    y_pred_class = np.argmax(y_pred, axis=1)

                    f1_macro_list.append(f1_score(y_true, y_pred_class, average='macro'))
                    f1_weighted_list.append(f1_score(y_true, y_pred_class, average='weighted'))
                    precision_list.append(precision_score(y_true, y_pred_class, average='macro', zero_division=0))
                    recall_list.append(recall_score(y_true, y_pred_class, average='macro', zero_division=0))

                resultados.append({
                    'modelo': nombre_modelo,
                    'tolerancia': tolerancia,
                    'num_layers': num_layers,
                    'units_per_layer': str(units_list),
                    'f1_macro': np.mean(f1_macro_list),
                    'f1_weighted': np.mean(f1_weighted_list),
                    'precision': np.mean(precision_list),
                    'recall': np.mean(recall_list)
                })

# Guardar resultados
import seaborn as sns
import matplotlib.pyplot as plt

df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel("/content/resultados_comparacion_modelos_tolerancia_kfold.xlsx", index=False)

# Mejor configuración por modelo (mantiene todas las columnas, incluidas listas)
top_idx = df_resultados.groupby('modelo')['f1_macro'].idxmax().dropna().astype(int)
top_config = df_resultados.loc[top_idx].copy().reset_index(drop=True)
top_config.to_excel("/content/top_combinaciones_f1_kfold.xlsx", index=False)

# Matrices de confusión para mejores combinaciones
for _, row in top_config.iterrows():
    if 'confusion_matrix' in row and isinstance(row['confusion_matrix'], list):
        cm = np.array(row['confusion_matrix'])
        modelo = row['modelo']
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=clases_unicas, yticklabels=clases_unicas, cbar=True, square=True, linewidths=0)
        plt.title(f"Matriz de Confusión Promedio (5-Fold): {modelo}")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.tight_layout()
        plt.savefig(f"/content/matriz_confusion_{modelo}_kfold.png")
        plt.show()

# Gráficas comparativas
sns.set(style="whitegrid")
for metric in ['f1_macro', 'f1_weighted', 'precision', 'recall']:
    g = sns.catplot(
        data=df_resultados,
        kind='bar',
        x='num_layers',
        y=metric,
        hue='units_per_layer',
        col='modelo',
        height=6,
        aspect=1.2
    )
    g.fig.subplots_adjust(top=0.85)
    g.fig.suptitle(f'{metric.replace("_", " ").title()} por número de capas, estructura y tolerancia (5-Fold)')
    g.set_axis_labels("Nº de capas", metric.replace("_", " ").title())
    g._legend.set_title("Estructura de neuronas")
    plt.savefig(f"/content/{metric}_comparacion_modelos_tolerancia_kfold.png")
    plt.show()

print("Evaluación con validación cruzada completada, matrices de confusión generadas y resultados exportados.")